In [ ]:
import numpy as np
from datetime import datetime, date, time, timedelta
from dateutil import parser
import csv

The objective of the deep learning system investigated in this project is the forecast of the N-S component of the IMF, Bz, measured in the GSM coordinate system at Lagrange point 1 (L1), in a two day window, 3 - 5 days ahead of the corresponding Solar and Heliophysics Observatory (SOHO) image products.
This challenge is cast as a binary classification problem and compared with a Gaussian Naive Bayes classifier baseline. 
The value computed is not the raw Bz from OMNI but rather we define the new Bz as the min of raw Bz minus the mean of raw Bz in the above mentioned two-day window.
The ground truth labels are obtained from the low-resolution, hourly-averaged values are obtained from the **OMNI** database: [OMNI2](https://omniweb.gsfc.nasa.gov/html/ow_data.html). The time range from this OMNI set that has been made available here is from Jan. 1st 1999 till Dec. 31st 2010. 

Provide the path to the downloaded OMNI data set to the ```np.genfromtxt()``` function in the cell below. 
The following definition for the 'nulls' variable comes from Section 3 of '**_Description of records and words_**' from the 'Fill values' column. There are a total of 57 columns listed but only 55 columns are actually present since 'F9.6' and 'F7.4' (columns 55 and 56, respectively) are omitted in the data set. Descriptions of the data are provided at the above NASA url. This step is performed to homogenize the representations of all values that are missing in the data set. Although this step is not strictly needed for our use-case here, it is useful if the user wished to use a different variable as their ground truth label. 

In [ ]:
PATH_TO_OMNI_DATA = '/home/carl/Documents/OMNI_DATA/'
omni2_data_raw = np.genfromtxt(f'{PATH_TO_OMNI_DATA}omni2_1999-01-01_to_2010-12-31.txt')
nulls = np.array([np.nan, np.nan, np.nan, 9999, 99, 99, 999, 999, 999.9, 999.9, 999.9, 999.9,
    999.9, 999.9, 999.9, 999.9, 999.9, 999.9, 999.9, 999.9, 999.9, 999.9,
    9999999., 999.9, 9999., 999.9, 999.9, 9.999, 99.99, 9999999., 999.9, 9999.,
    999.9, 999.9, 9.999, 999.99, 999.99, 999.9, 99, 999, 99999, 9999,
    999999.99, 99999.99, 99999.99, 99999.99, 99999.99, 99999.99, 0, 999, 999.9,
    999.9, 99999, 99999, 99.9
])
print('len(nulls):', len(nulls))
omni2_data_nan_processed = np.where(omni2_data_raw == nulls[None, :], np.nan, omni2_data_raw)
print('shape(omni2_data_nan_processed):', np.shape(omni2_data_nan_processed))

Convert OMNI date-time format of 'Year-Day_of_Year-Time_of_Day' to date-time object.

Choose the Bz GSM column which is the 16th column (counting the pythonic way from 0)

Identify dates for which the ground truth label is missing.

**Note**: If there would be missing ground truth labels that would influence our approach, we would need to account for this by removing corresponding SOHO images from the data cubes. From the procedure used in this project to arrive at the computed Bz value, these missing raw Bz values do not require any modification to the pipeline produced image cubes in this case. In another ground truth label setting or forecasting window, they might.  

In [ ]:
omni2_date = [datetime(int(yr),1,1) + timedelta(days = int(day) - 1) + timedelta(hours = hr) for yr,day,hr in np.vstack((omni2_data_nan_processed.T[0],omni2_data_nan_processed.T[1], omni2_data_nan_processed.T[2])).T ]

omni2_Bz = omni2_data_nan_processed.T[16]

missing_Bz_vals_ind = np.where(np.array(omni2_Bz) != np.array(omni2_Bz))[0]
print('Missing ground truth labels from Jan. 1st 1999 to Dec. 31st 2010 in data:\n', [str(omni2_date[ind]) for ind in missing_Bz_vals_ind])
print('Number of missing ground truth labels:', len(missing_Bz_vals_ind))

Convert above transformed OMNI date-time object to a string for efficient matching when comparing with synced string date-times.

In [ ]:
omni2_date_copy_str = [str(elem) for elem in omni2_date]

Provide the path to the synced times contained in the **CSV** file for which to obtain the ground truth labels and specify ```dtype = 'str'```. 
As a check, ensure that the output length corresponds to the length of the CSV file which you expect to have.  

In [ ]:
PATH_TO_SYNCED_TIMES = '/home/carl/Documents/synced_csvs_1_3_7fams/'
synced_times = np.genfromtxt(f'{PATH_TO_SYNCED_TIMES}1999-01-01_to_2010-12-31_MDI_96m_6_6_128_times_sync_onlyMDI.csv',dtype = 'str')
print('len(synced_times):', len(synced_times))

Convert '19990202160302' string date-time format to a date-time object and compute the value of the date-time object 3 days ahead of the time found after rounding to the nearest hour and convert to string. Rounding is done to enable direct comparison with the OMNI date-time objects.

In [ ]:
synced_times_datetimes = [parser.parse(elem) for elem in synced_times]
synced_times_datetimes_rounded = [elem.replace(second = 0, microsecond = 0, minute = 0, hour = elem.hour) + timedelta(hours = elem.minute//30) for elem in synced_times_datetimes]
synced_times_datetimes_rounded_3days_ahead = np.array(synced_times_datetimes_rounded) + timedelta(days = 3)
synced_times_datetimes_rounded_3days_ahead_copy_str = [str(elem) for elem in synced_times_datetimes_rounded_3days_ahead]

Now can compare synced times directly with the times from OMNI data and compute Bmin - Bmean, 3-5 days ahead of the SOHO image product times. The figure of 48 is added in order to arrive at the 5 day mark of the two-day window. Need to use np.nanmin and np.nanmean as a result of the 24 missing raw Bz values. This is the most time consuming step thus far. 

In [ ]:
omni2_3days_ahead_ind = [np.where(np.array(omni2_date_copy_str) == elem)[0][0] for elem in synced_times_datetimes_rounded_3days_ahead_copy_str]
omni2_Bzmin_minus_mean_3to5days_ahead = [ np.round(np.nanmin(omni2_Bz[elem: elem + 48]) - np.nanmean(omni2_Bz[elem: elem + 48]),2) for elem in omni2_3days_ahead_ind]

Option to output the computed Bz ground truth labels corresponding to the synced times to a new CSV file in the same directory as the synced times. The user can change the name for the CSV file that is to be output.

In [ ]:
with open(f'{PATH_TO_SYNCED_TIMES}3products_Bz_min_minus_mean_3to5days_ahead.csv', 'a') as f:
    writer = csv.writer(f, delimiter='\n')
    writer.writerow(omni2_Bzmin_minus_mean_3to5days_ahead)